We want to make a turn-based strategy game, based on a **hexagonal grid**.

Wait, how do we even lay out a hexagonal grid in kivy?

 http://playtechs.blogspot.com/2007/04/hex-grids.html
 
 Also
 
 http://gamedev.stackexchange.com/questions/15881/hexagonal-grid-tiles-tutorials
 

Ignore device orientation.

Let's lay out a grid. First we need to draw hexagons. Clone gamecamp. change the object names.
Can we lay out the basic screen?

In [ ]:
%%file strategygame.kv

#:include debug.kv

<StrategyGame>:
    BoxLayout:
        orientation: 'horizontal'
        DebugLabel:
            text: 'Main map'
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33


In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout


class StrategyGame(FloatLayout):
    pass


class StrategyGameApp(App):
    def build(self):
        return StrategyGame()

if __name__ == '__main__':
    StrategyGameApp().run()


In [ ]:
#!python main.py


Let's make a gridlayout in the game area, and give it a label so we can refer to it from our code. We are going to build a rectangular grid

In [ ]:
%%file strategygame.kv

#:include debug.kv

<StrategyGame>:
    main_map: _main_map
    BoxLayout:
        orientation: 'horizontal'
        GridLayout:
            id: _main_map
            cols: 10
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33


Now let's hook to that from code. We will do this in `__init__` in our main StrategyGame class. Let's make a loop to add buttons to the root widget. We will use properties in the kv file to determine how many rows/cols to add. We will also need to refer to various objects in the widget hierarchy in the code.

You can refer to the top-level object (non-indented) as `root` in the kv file, but it we want to refer to conveniently elsewhere in the code, we need to use an id.

`id: _game`

and then from any widget we want to refer to it, we do a 

`game: _game`

We'll use this to access the number of regions to layout in the grid via `map_cols` and `map_rows` from the main game widget.  

Fill in grids: it fills it in by default, left to right, top to bottom, and give them coordinate references. 

In [ ]:
%%file strategygame.kv

#:include debug.kv

<StrategyGame>:
    id: _game
    main_map: _main_map
    map_rows: 10
    map_cols: 10
    BoxLayout:
        orientation: 'horizontal'
        GridLayout:
            id: _main_map
            game: _game
            cols: root.map_cols
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33


In [ ]:
# %load main.py
from kivy.app import App
from kivy import properties
from kivy.uix import button
from kivy.uix.floatlayout import FloatLayout


class StrategyGame(FloatLayout):
    main_map = properties.ObjectProperty(None)
    map_rows = properties.NumericProperty(0)
    map_cols = properties.NumericProperty(0)

    def __init__(self, **kwargs):
        super(StrategyGame, self).__init__(**kwargs)

        number_of_regions = self.map_rows * self.map_cols
        for region in xrange(0, number_of_regions):
            row = region / self.map_cols
            col = region % self.map_cols
            self.main_map.add_widget(button.Button(text='({}, {})'.format(row, col)))


class StrategyGameApp(App):
    def build(self):
        return StrategyGame()

if __name__ == '__main__':
    StrategyGameApp().run()


We draw alternating rows with the patterns:
```
/ \
| |
| |
```
and
```
\ /
 |
 |
```

In [ ]:
%%file strategygame.kv

#:include debug.kv

<StrategyGame>:
    id: _game
    main_map: _main_map
    map_rows: 10
    map_cols: 10
    BoxLayout:
        orientation: 'horizontal'
        GridLayout:
            id: _main_map
            game: _game
            cols: root.map_cols
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33


In [ ]:
%%file main.py
from kivy.app import App
from kivy import properties
from kivy.uix import button
from kivy.uix.floatlayout import FloatLayout


class StrategyGame(FloatLayout):
    main_map = properties.ObjectProperty(None)
    map_rows = properties.NumericProperty(0)
    map_cols = properties.NumericProperty(0)

    def __init__(self, **kwargs):
        super(StrategyGame, self).__init__(**kwargs)

        number_of_regions = self.map_rows * self.map_cols
        for region in xrange(0, number_of_regions):
            row = region / self.map_cols
            col = region % self.map_cols
            self.main_map.add_widget(button.Button(text='({}, {})'.format(row, col)))


class StrategyGameApp(App):
    def build(self):
        return StrategyGame()

if __name__ == '__main__':
    StrategyGameApp().run()


Wait, that doesn't draw where we think it should.'

In [ ]:
%%file main.py
import collections

from kivy.app import App
from kivy import properties
from kivy import graphics
from kivy.uix import label
from kivy.uix.floatlayout import FloatLayout


MapCoords = collections.namedtuple('MapCoords', ['row', 'col'])


class StrategyGame(FloatLayout):
    main_map = properties.ObjectProperty(None)
    map_rows = properties.NumericProperty(0)
    map_cols = properties.NumericProperty(0)

    def __init__(self, **kwargs):
        super(StrategyGame, self).__init__(**kwargs)

        number_of_regions = self.map_rows * self.map_cols
        for region in xrange(0, number_of_regions):
            row = region / self.map_cols
            col = region % self.map_cols
            self.main_map.add_widget(HexMapCell(row=row, col=col))


class HexMapCell(label.Label):
    def __init__(self, row=0, col=0, **kwargs):
        self.region_in_map = MapCoords(row, col)
        super(HexMapCell, self).__init__(**kwargs)
        self.draw_hex_edge()

    def draw_hex_edge(self):
        edge = ''
        if self.region_in_map.col % 2 == 0:
            row_mod = self.region_in_map.row % 6
            if row_mod == 0:
                edge = 'BU'
                with self.canvas:
                    graphics.Color(1, 1, 1, 1)
                    graphics.Line(points=[self.x, self.y, self.width + self.x, self.height + self.y])
            elif row_mod in (1, 2):
                edge = 'L '
            elif row_mod == 3:
                edge = 'TD'
            elif row_mod in (4, 5):
                edge = ' R'
        else:
            row_mod = self.region_in_map.row % 6
            if row_mod == 0:
                edge = 'TD'
            elif row_mod in (1, 2):
                edge = ' R'
            elif row_mod == 3:
                edge = 'BU'
            elif row_mod in (4, 5):
                edge = 'L '

        self.text = edge

class StrategyGameApp(App):
    def build(self):
        return StrategyGame()

if __name__ == '__main__':
    StrategyGameApp().run()


In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.gridlayout import GridLayout
from kivy.lang import Builder
from kivy.properties import NumericProperty

class HelloWorld(GridLayout):
    cols = NumericProperty(4)
    def __init__(self, **kw):
        super(HelloWorld, self).__init__(**kw)
        self.add_widget(self.BU())
        self.add_widget(self.TD())
        self.add_widget(self.BU())
        self.add_widget(self.TD())

    def BU(self):
        return Builder.load_string('''
Label:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.y, self.right, self.top)
            width: 2''')

    def TD(self):
        return Builder.load_string('''
Label:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.top, self.right, self.y)
            width: 2''')

    def L(self):
        return Builder.load_string('''
Label:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.y, self.x, self.top)
            width: 2''')

    def R(self):
        return Builder.load_string('''
Label:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.right, self.y, self.right, self.top)
            width: 2''')


class HelloWorldApp(App):
    def build(self):
        return HelloWorld()

if __name__ == '__main__':
    HelloWorldApp().run()


In [ ]:
# %load strategygame.kv

#:include debug.kv

<StrategyGame>:
    id: _game
    main_map: _main_map
    map_rows: 10
    map_cols: 10
    BoxLayout:
        orientation: 'horizontal'
        GridLayout:
            id: _main_map
            game: _game
            cols: root.map_cols
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33


## Trying to draw the hexmap

We tried adding the ability to style our widgets for to draw the Hexmap using lines like this

    BU: /
    TD: \
    L: |
    R:   |
We get hexs now, but we need to figure out how to do the aspect ratio properly now...for a spiky top, we need each grid widget to have aspec ratio $$\sqrt{3}\cdot height = width$$

In [ ]:
%%file strategygame.kv

#:include debug.kv

<StrategyGame>:
    id: _game
    main_map: _main_map
    map_rows: 10
    map_cols: 10
    BoxLayout:
        orientation: 'horizontal'
        GridLayout:
            id: _main_map
            game: _game
            cols: root.map_cols
            size_hint: .75, 1
        BoxLayout:
            orientation: 'vertical'
            size_hint: .25, 1
            DebugLabel:
                text: 'status'
                size_hint: 1, .66
            DebugLabel:
                text: 'mini-map'
                size_hint: 1, .33

<BU>:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.y, self.right, self.top)
            width: 2
<TD>:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.top, self.right, self.y)
            width: 2
<L>:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.x, self.y, self.x, self.top)
            width: 2
<R>:
    canvas:
        Color:
            rgba: (1,1,1,1)
        Line:
            points: (self.right, self.y, self.right, self.top)
            width: 2


In [ ]:
%%file main.py
import collections

from kivy.app import App
from kivy import properties
from kivy import graphics
from kivy.uix import label
from kivy.uix.floatlayout import FloatLayout
import math

MapCoords = collections.namedtuple('MapCoords', ['row', 'col'])


class StrategyGame(FloatLayout):
    main_map = properties.ObjectProperty(None)
    map_rows = properties.NumericProperty(0)
    map_cols = properties.NumericProperty(0)

    def __init__(self, **kwargs):
        super(StrategyGame, self).__init__(**kwargs)

        number_of_regions = self.map_rows * self.map_cols
        for region in xrange(0, number_of_regions):
            row = region / self.map_cols
            col = region % self.map_cols
            self.main_map.add_widget(self.pick_hex_cell(row=row, col=col))


    def pick_hex_cell(self, row, col):
        row_mod = row % 6
        if col % 2 == 0:
            if row_mod == 0:
                return BU()
            elif row_mod in (1, 2):
                return L()
            elif row_mod == 3:
                return TD()
            elif row_mod in (4, 5):
                return R()
        else:
            if row_mod == 0:
                return TD()
            elif row_mod in (1, 2):
                return R()
            elif row_mod == 3:
                return BU()
            elif row_mod in (4, 5):
                return L()


class HexMapCell(label.Label):
    def __init__(self, row=0, col=0, **kwargs):
        self.region_in_map = MapCoords(row, col)
        super(HexMapCell, self).__init__(**kwargs)

class BU(HexMapCell):
    pass
class TD(HexMapCell):
    pass
class L(HexMapCell):
    pass
class R(HexMapCell):
    pass


class StrategyGameApp(App):
    def build(self):
        return StrategyGame()

if __name__ == '__main__':
    StrategyGameApp().run()
